In [87]:
import os 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import h5py
import numpy as np
import glob
import shutil
matplotlib.rcParams['figure.figsize'] = [15, 15]
pd.options.mode.chained_assignment = None

# Link each TNEID to a molecular archetype

In [2]:
# data from : /data/lungNENomics/work/Descriptive_manuscript_data/MOFA/MOFA.Exp.Meth.Alt.CNV_lungNENomicsCombined/ParetoTI/variables_archetypes_MOFA_LNET.RData
archetype = pd.read_csv("Mofa_partoTI_archetypes.csv")
archetype.head()

,Unnamed: 0,manuscript_id,sample_id,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,...,PFS_census_status,PFS_months,Alcala2019_cluster_LNET,Alcala2019_cluster_LNEN,published_cluster,published_cluster_v2,archetype_k3_LF2,archetype_k3_LF2_label,archetype_k4_LF3,archetype_k4_LF3_label
0,1,LNEN002,LNEN002,2.931527,-0.778837,-2.822321,-0.005001,0.654035,0.472506,-0.338176,...,NaN,NaN,Ca B,Ca B,Ca B,Ca B,V2,Ca B,V3,Ca B
1,2,LNEN003,LNEN003,0.141281,1.908876,-0.558505,0.350397,0.381332,0.391136,-0.052032,...,NaN,NaN,Ca A2,Ca A,Ca A2,Ca A2,V3,Ca A2,V4,Ca A2
2,3,LNEN004,LNEN004,0.121415,2.148833,-0.492770,0.273303,0.423751,-0.055546,-0.096825,...,NaN,NaN,Ca A2,Ca A,Ca A2,Ca A2,V3,Ca A2,V4,Ca A2
3,4,LNEN005,LNEN005,-0.053188,-0.663662,0.872511,0.665158,-5.139342,1.168464,0.925062,...,NaN,NaN,Ca A1,LCNEC,supra-carcinoid,supra-carcinoid,V1,Ca A1,V1,sc-enriched
4,5,LNEN006,LNEN006,0.290015,1.270474,0.670284,-0.559943,0.224200,-0.206417,-0.091758,...,NaN,NaN,Ca A2,Ca A,Ca A2,Ca A2,V3,Ca A2,V4,Ca A2


In [17]:
archetype.shape

(319, 46)

In [3]:
# Get TNE_ID with Barlow-Twins projections
BT_all_proj = pd.read_csv("/home/mathiane/LNENWork/LNEN_Molecular_groups_barlow_twin/projector/barlow_twins_encoded_vectors_scale_path_review.csv")

In [18]:
BT_all_proj.head()

,Unnamed: 0,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X124,X125,X126,X127,img_id,tne_id,img_id_c,tne_id_c,x,y
0,1,0.010731,-0.017492,-0.053791,0.006058,-0.040010,0.006106,-0.024915,-0.011729,0.007921,...,0.002239,0.007743,0.008007,-0.016012,TNE1019_30721_19585,TNE1019,TNE1019_30721_19585,TNE1019,30721.0,19585
1,2,0.003174,-0.002447,-0.040421,0.000079,-0.030701,0.005880,-0.027878,-0.006616,0.011590,...,-0.013962,0.007799,0.012292,0.005784,TNE1019_33409_28801,TNE1019,TNE1019_33409_28801,TNE1019,33409.0,28801
2,3,-0.010324,-0.008172,-0.039857,-0.003609,-0.023241,-0.007832,-0.032735,-0.020779,0.005676,...,-0.001943,0.003459,0.009208,0.010942,TNE1019_45313_14209,TNE1019,TNE1019_45313_14209,TNE1019,45313.0,14209
3,4,0.011308,-0.018657,-0.060345,0.015401,-0.023393,-0.007311,-0.022321,-0.004438,0.009153,...,0.011426,0.008252,0.012065,-0.010705,TNE1019_9985_23809,TNE1019,TNE1019_9985_23809,TNE1019,9985.0,23809
4,5,-0.000423,-0.025801,-0.052976,0.005961,-0.026089,-0.010742,-0.021492,-0.003688,0.008236,...,-0.000672,0.009480,0.010750,-0.007972,TNE1019_15361_11905,TNE1019,TNE1019_15361_11905,TNE1019,15361.0,11905


In [19]:
len(set(BT_all_proj["tne_id_c"]))

311

In [20]:
## Load clinical data
clinical_df = pd.read_excel('/data/lungNENomics/work/Descriptive_manuscript_data/Clinical_data/lungNENomics_clinical_v39.xlsx', index_col=0, skiprows=[0],
                            engine='openpyxl') 
clinical_df.head()

,manuscript_id,lungNENomics_id,case_number_path_review,centre,centre_corrected,sex,AGE-DIAGNOSIS,AGE.at.DIAGNOSIS,age_calculated,age_corrected,...,cause_of_death,census_date,census_status,overall_survival_years,overall_survival_months,PFS_census_date,PFS_census_status,PFS_years,PFS_months,Notes
centre_patient_id,,,,,,,,,,,,,,,,,,,,,
10EX00072,LNEN124,LNEN124,33,Bron,Bron,F,50.0,50.0,51.0,51,...,NaN,2010-12-13,alive,0.112329,1.347945,NaT,NaN,NaN,NaN,NaN
11EX00042,LNEN125,LNEN125,34,Bron,Bron,M,44.0,44.0,44.0,44,...,NaN,2018-09-20,alive,7.400000,88.800000,2018-09-20,no,7.400000,88.800000,NaN
11EX00090,LNEN126,LNEN126,35,Bron,Bron,F,79.0,79.0,79.0,79,...,NaN,2018-05-03,alive,6.693151,80.317808,2018-05-03,no,6.693151,80.317808,NaN
11EX00105,LNEN127,LNEN127,36,Bron,Bron,F,38.0,38.0,39.0,39,...,NaN,2011-11-07,alive,0.126027,1.512329,NaT,NaN,NaN,NaN,NaN
11EX00143,LNEN128,LNEN128,37,Bron,Bron,M,43.0,43.0,43.0,43,...,NaN,2015-12-09,alive,3.986301,47.835616,NaT,NaN,NaN,NaN,NaN


In [21]:
clinical_df['centre_patient_id'] = clinical_df.index

In [22]:
## Get patients in WSI data base
all_wsi = glob.glob("/home/mathiane/LNEN_img_db/*/HES/Leica/x40/*/*/*")
centre = []
staining = []
scanner = []
mag = []
patient_id = []
tne_id = []
for ele in all_wsi:
    wsi_details = ele.split("/")
    centre.append(wsi_details[4])
    staining.append(wsi_details[5])
    scanner.append(wsi_details[6])
    mag.append(wsi_details[7])
    patient_id.append(wsi_details[8])
    tne_id.append(wsi_details[9])
WSI_df = pd.DataFrame({"centre" : centre, "staining": staining, "scanner":scanner,
                       "mag":mag, "patient_id":patient_id, "tne_id" : tne_id})

In [23]:
sum(WSI_df.duplicated())
WSI_df = WSI_df.drop_duplicates()

In [24]:
WSI_df.shape

(311, 6)

In [25]:
## Restrict research in df to TNEID with BT proj
WSI_df = WSI_df[WSI_df["tne_id"].isin(list(BT_all_proj["tne_id_c"]))]

In [26]:
## Get patients with multiples WSI
patients_with_several_wsi = WSI_df[WSI_df.duplicated("patient_id")]["patient_id"]
len(patients_with_several_wsi)


38

In [27]:
patients_with_several_wsi

63       ML132664
65       ML134872
106      ML123826
110          CA86
111          CA86
114          CA42
116          CA10
119          CA77
122          CA84
125          CA78
130          CA41
131          CA41
134          CA69
137     19.15936A
178     000585425
181     000422868
193     000523215
204     000441557
217     000530926
240     3369.14.3
242      22038.17
246       9754.17
247       9754.17
248       9754.17
251      11543.15
253    11960.14.3
255      26524.16
258       3613.18
261     2554.14.3
264       5368.17
267      22890.17
269       5883.16
273     2439.14.3
275      19989.16
277      26153.16
279       6324.17
309    AN14006022
311    AN16003873
Name: patient_id, dtype: object

In [28]:
WSI_df[WSI_df["patient_id"] == "AN14006022"]

,centre,staining,scanner,mag,patient_id,tne_id
308,Lyon-CLB,HES,Leica,x40,AN14006022,TNE1403
309,Lyon-CLB,HES,Leica,x40,AN14006022,TNE1404


In [29]:
## Load path review to select TNEID included in path-review for patients with several WSIs
path_review = pd.read_excel('/home/mathiane/LNENWork/ESMOOpenFinalScripts/data/pathological_review_concensus_data_2112022.xlsx', index_col=0, skiprows=[0],
                            engine='openpyxl') 
path_review.head()
tne_id_HES = path_review[path_review["material"]=="HES"]["TNEID"]

In [35]:
c = 0 
tne_to_ignore = [] # For patients with several HE at mag 40 that were include in the path-review we ignore the WSI that were not used by pathologists
for tne_path_review in tne_id_HES:
    tne_patientid = WSI_df[WSI_df["tne_id"] == tne_path_review]["patient_id"].values[0]
    if tne_patientid in list(patients_with_several_wsi):
        # TNE to ignore
        tne_to_ignore.append(WSI_df[(WSI_df["patient_id"] == tne_patientid) & (~WSI_df["tne_id"].isin(tne_id_HES))]["tne_id"].values[0])
        c += 1
print(c) # Conclusion 34 patients can be selected like this 

34


In [36]:
## Between TNE2142 an TNE2141:
## Choice TNE2142 => Bigger more context warning not the same block!
to_ingore = ["TNE2141"]
## Between TNE0797 an TNE2104:
## Choice TNE0797 => TNE2104 roughly no tumour tissue
to_ingore.append("TNE2104")
## Between TNE1496, TNE0590, TNE0589
## Ignore TNE0590 => few tissue
## Note TNE1496 and TNE090 | TNE0589 not the same block 
## Random choice
to_ingore.append("TNE0590")
to_ingore.append("TNE0589")
# WARNING : DEAL WITH THIS INFO LATER

In [37]:
print(WSI_df.shape)
WSI_df = WSI_df[~WSI_df["tne_id"].isin(tne_to_ignore)]
print(WSI_df.shape)

(308, 6)
(274, 6)


In [38]:
## Get remaining patient ID with several WSIs
patient_id_several_wsi_not_in_path_review = WSI_df[WSI_df.duplicated("patient_id")]["patient_id"]

In [39]:
for patient in list(patient_id_several_wsi_not_in_path_review):
    print(WSI_df[WSI_df["patient_id"]==patient])
    print("---------------------------------")

    centre staining scanner  mag patient_id   tne_id
109  Nancy      HES   Leica  x40       CA86  TNE2141
111  Nancy      HES   Leica  x40       CA86  TNE2142
---------------------------------
    centre staining scanner  mag patient_id   tne_id
130  Nancy      HES   Leica  x40       CA41  TNE0797
131  Nancy      HES   Leica  x40       CA41  TNE2104
---------------------------------
    centre staining scanner  mag patient_id   tne_id
246  Turin      HES   Leica  x40    9754.17  TNE1496
247  Turin      HES   Leica  x40    9754.17  TNE0590
248  Turin      HES   Leica  x40    9754.17  TNE0589
---------------------------------
    centre staining scanner  mag patient_id   tne_id
246  Turin      HES   Leica  x40    9754.17  TNE1496
247  Turin      HES   Leica  x40    9754.17  TNE0590
248  Turin      HES   Leica  x40    9754.17  TNE0589
---------------------------------


In [40]:
print(WSI_df.shape)
WSI_df = WSI_df[~WSI_df["tne_id"].isin(to_ingore)]
print(WSI_df.shape)

(274, 6)
(270, 6)


In [41]:
patient_id_corrected  = []
for patient_id in list(WSI_df["patient_id"]):
    if patient_id.find("LCCH") != -1:
        print(patient_id, "LCCH-" + patient_id.split("-")[1])
        patient_id_corrected.append("LCCH-" + patient_id.split("-")[1])
    
    elif patient_id == "19.15936A":
        print(patient_id, "19.15936 A- A25")
        patient_id_corrected.append("19.15936 A- A25")
        
    elif patient_id.find(".") != -1 :
        print(patient_id, patient_id.replace(".", "/"))
        patient_id_corrected.append(patient_id.replace(".", "/"))
        
    elif patient_id == "AN14018907" :
        print(patient_id, "1424863")
        patient_id_corrected.append("1424863")
        
    elif patient_id == "AN16006491":
        print(patient_id, "1607106")
        patient_id_corrected.append("1607106")

        
    elif patient_id == "AN15012673":
        print(patient_id, "1515223")
        patient_id_corrected.append("1515223")

    elif patient_id == "AN14006022":
        print(patient_id , "1316113")
        patient_id_corrected.append("1316113")
        
    elif patient_id == "AN16003873":
        print(patient_id , "1518417")
        patient_id_corrected.append("1518417")
        
    elif patient_id == "AN16009141":
        print(patient_id , "1609329")
        patient_id_corrected.append("1609329")
        
    elif patient_id == "46404_1997":
        print(patient_id, "46404/1997")
        patient_id_corrected.append("46404/1997")
        
    elif patient_id == "1594_2009":
        print(patient_id, "1594/2009")
        patient_id_corrected.append("1594/2009")
        
    elif patient_id == "92276":
        print(patient_id, "000092276")
        patient_id_corrected.append("000092276")
        
    elif patient_id == "40072_1997":
        print(patient_id, "40072/1997")
        patient_id_corrected.append("40072/1997")


    else:
        patient_id_corrected.append(patient_id)

WSI_df["patient_id_corrected"] = patient_id_corrected

17.2293 17/2293
19.15936A 19.15936 A- A25
92276 000092276
LCCH01-697 LCCH-697
LCCH01-640 LCCH-640
LCCH01-831 LCCH-831
LCCH01-703 LCCH-703
LCCH01-478 LCCH-478
LCCH01-230 LCCH-230
34524.2010 34524/2010
134279.2009 134279/2009
46404_1997 46404/1997
21399.2009 21399/2009
40072_1997 40072/1997
47915.2009 47915/2009
1594_2009 1594/2009
14462.2010 14462/2010
3369.14.3 3369/14/3
22038.17 22038/17
9754.17 9754/17
19135.17 19135/17
11543.15 11543/15
11960.14.3 11960/14/3
26524.16 26524/16
17534.15 17534/15
3613.18 3613/18
2554.14.3 2554/14/3
2943.14.3 2943/14/3
5368.17 5368/17
2638.16 2638/16
22890.17 22890/17
5883.16 5883/16
24258.17 24258/17
2439.14.3 2439/14/3
19989.16 19989/16
26153.16 26153/16
6324.17 6324/17
AN14018907 1424863
AN16006491 1607106
AN15012673 1515223
AN14006022 1316113
AN16003873 1518417
AN16009141 1609329


In [42]:
len(set(WSI_df["patient_id_corrected"]))

270

In [43]:
clinical_df.index = clinical_df.index.astype('str')

In [44]:
clinical_df["centre_patient_id_col"] = clinical_df.index

In [45]:
WSI_df_clinical_tmp2 = WSI_df.merge(clinical_df, left_on='patient_id_corrected', right_on='centre_patient_id_col', how='inner')

In [46]:
WSI_df_clinical_tmp2.shape

(264, 90)

In [47]:
WSI_df[~WSI_df["patient_id_corrected"].isin(list(WSI_df_clinical_tmp2["centre_patient_id_col"])) ]

,centre,staining,scanner,mag,patient_id,tne_id,patient_id_corrected
112,Nancy,HES,Leica,x40,19B1596,TNE2158,19B1596
117,Nancy,HES,Leica,x40,12H02399_T2,TNE0342,12H02399_T2
120,Nancy,HES,Leica,x40,09H5932_B,TNE0221,09H5932_B
126,Nancy,HES,Leica,x40,17.2293,TNE2155,17/2293
127,Nancy,HES,Leica,x40,09H5932_A,TNE0217,09H5932_A
132,Nancy,HES,Leica,x40,12H02399_T5,TNE0345,12H02399_T5


In [48]:
WSI_df_clinical = WSI_df_clinical_tmp2

In [49]:
print(WSI_df_clinical.shape, WSI_df_clinical[WSI_df_clinical["manuscript_id"] != "."].shape)

(264, 90) (233, 90)


In [50]:
print(WSI_df_clinical.shape,
WSI_df_clinical.merge(archetype, on='manuscript_id', how='inner').shape)

(264, 90) (185, 135)


In [51]:
WSI_df_clinical_with_manuscript_id = WSI_df_clinical[WSI_df_clinical["manuscript_id"] != "."]

In [52]:
WSI_df_clinical_arch = WSI_df_clinical.merge(archetype, on='manuscript_id', how='inner')

In [53]:
set(WSI_df_clinical_with_manuscript_id["manuscript_id"]) - set(WSI_df_clinical_arch["manuscript_id"])

{'LNEN045',
 'LNEN046',
 'LNEN049',
 'LNEN059',
 'LNEN062',
 'LNEN070',
 'LNEN075',
 'LNEN078',
 'LNEN080',
 'LNEN088',
 'LNEN120',
 'LNEN124',
 'LNEN138',
 'LNEN141',
 'LNEN144',
 'LNEN149',
 'LNEN150',
 'LNEN160',
 'LNEN186',
 'LNEN189',
 'LNEN200',
 'LNEN222',
 'LNEN242',
 'LNEN254',
 'LNEN256',
 'LNEN257',
 'LNEN259',
 'LNEN261',
 'LNEN262',
 'LNEN264',
 'LNEN265',
 'LNEN266',
 'LNEN267',
 'LNEN271',
 'LNEN275',
 'LNEN278s2',
 'S00076-B',
 'S00089-B',
 'S00094-B',
 'S00118-B',
 'S00128-B',
 'S01103-B',
 'S01493-B',
 'S01501-B',
 'S01502-B',
 'S01504-B',
 'S01605-B',
 'S01666-B'}

In [54]:
len(set(WSI_df_clinical_with_manuscript_id["manuscript_id"]) - set(WSI_df_clinical_arch["manuscript_id"]))

48

In [55]:
WSI_df_clinical_arch.head()

,centre_x,staining,scanner,mag,patient_id,tne_id,patient_id_corrected,manuscript_id,lungNENomics_id,case_number_path_review,...,PFS_census_status_y,PFS_months_y,Alcala2019_cluster_LNET,Alcala2019_cluster_LNEN,published_cluster,published_cluster_v2,archetype_k3_LF2,archetype_k3_LF2_label,archetype_k4_LF3,archetype_k4_LF3_label
0,Nice,HES,Leica,x40,LB10-0505,TNE1428,LB10-0505,LNEN100,LNEN100,65,...,no,65.391781,NaN,NaN,NaN,NaN,V1,Ca A1,V2,Ca A1
1,Nice,HES,Leica,x40,LB13-0454,TNE1433,LB13-0454,LNEN106,LNEN106,70,...,no,55.068493,NaN,NaN,NaN,NaN,V2,Ca B,V3,Ca B
2,Nice,HES,Leica,x40,LB06-0162,TNE1412,LB06-0162,LNEN093,LNEN093,54,...,event,9.172603,NaN,NaN,NaN,NaN,V1,Ca A1,V2,Ca A1
3,Nice,HES,Leica,x40,LB13-0723,TNE1417,LB13-0723,LNEN107,LNEN107,59,...,no,30.115068,NaN,NaN,NaN,supra-carcinoid,V1,Ca A1,V1,sc-enriched
4,Nice,HES,Leica,x40,LB09-0132,TNE1415,LB09-0132,LNEN099,LNEN099,57,...,event,1.709589,NaN,NaN,NaN,NaN,V1,Ca A1,V2,Ca A1


In [56]:
WSI_df_clinical_arch.shape

(185, 135)

In [57]:
TNEID_arch  = WSI_df_clinical_arch[["tne_id","archetype_k4_LF3_label"]]

In [58]:
TNEID_arch.head()

,tne_id,archetype_k4_LF3_label
0,TNE1428,Ca A1
1,TNE1433,Ca B
2,TNE1412,Ca A1
3,TNE1417,sc-enriched
4,TNE1415,Ca A1


In [59]:
TNEID_arch= TNEID_arch.replace({'archetype_k4_LF3_label': {"Ca A1": "CaA1", "Ca A2": "CaA2", "Ca B": "CaB"}})

In [60]:
TNEID_arch.value_counts("archetype_k4_LF3_label")

archetype_k4_LF3_label
CaA1           72
CaA2           61
CaB            46
sc-enriched     6
dtype: int64

In [61]:
TNEID_arch.head()

,tne_id,archetype_k4_LF3_label
0,TNE1428,CaA1
1,TNE1433,CaB
2,TNE1412,CaA1
3,TNE1417,sc-enriched
4,TNE1415,CaA1


In [69]:
BTProjArch.head()

,Unnamed: 0,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X126,X127,img_id,tne_id_x,img_id_c,tne_id_c,x,y,tne_id_y,archetype_k4_LF3_label
0,1,0.010731,-0.017492,-0.053791,0.006058,-0.040010,0.006106,-0.024915,-0.011729,0.007921,...,0.008007,-0.016012,TNE1019_30721_19585,TNE1019,TNE1019_30721_19585,TNE1019,30721.0,19585,TNE1019,CaA2
1,2,0.003174,-0.002447,-0.040421,0.000079,-0.030701,0.005880,-0.027878,-0.006616,0.011590,...,0.012292,0.005784,TNE1019_33409_28801,TNE1019,TNE1019_33409_28801,TNE1019,33409.0,28801,TNE1019,CaA2
2,3,-0.010324,-0.008172,-0.039857,-0.003609,-0.023241,-0.007832,-0.032735,-0.020779,0.005676,...,0.009208,0.010942,TNE1019_45313_14209,TNE1019,TNE1019_45313_14209,TNE1019,45313.0,14209,TNE1019,CaA2
3,4,0.011308,-0.018657,-0.060345,0.015401,-0.023393,-0.007311,-0.022321,-0.004438,0.009153,...,0.012065,-0.010705,TNE1019_9985_23809,TNE1019,TNE1019_9985_23809,TNE1019,9985.0,23809,TNE1019,CaA2
4,5,-0.000423,-0.025801,-0.052976,0.005961,-0.026089,-0.010742,-0.021492,-0.003688,0.008236,...,0.010750,-0.007972,TNE1019_15361_11905,TNE1019,TNE1019_15361_11905,TNE1019,15361.0,11905,TNE1019,CaA2


## Create h5 files to train Roformer MIL

In [65]:
BTProjArch = BT_all_proj.merge(TNEID_arch, left_on='tne_id_c',  right_on='tne_id' , how='inner')
print(BT_all_proj.shape, BTProjArch.shape )

(5496736, 135) (3376729, 137)


In [70]:
BTProjArch  = BTProjArch.rename(columns={"archetype_k4_LF3_label": "arch"})
BTProjArchCaA1CaA2 = BTProjArch[BTProjArch["arch"].isin(["CaA1", "CaA2"])]

In [71]:
BTProjArchCaA1CaA2.shape

(2301868, 137)

In [78]:
def save_hdf5(output_path, asset_dict, attr_dict=None, mode="a"):
    file = h5py.File(output_path, mode)
    for key, val in asset_dict.items():
        data_shape = val.shape
        if key not in file:
            data_type = val.dtype
            chunk_shape = (1,) + data_shape[1:]
            maxshape = (None,) + data_shape[1:]
            dset = file.create_dataset(
                key,
                shape=data_shape,
                maxshape=maxshape,
                chunks=chunk_shape,
                dtype=data_type,
            )
            dset[:] = val
            if attr_dict is not None:
                if key in attr_dict.keys():
                    for attr_key, attr_val in attr_dict[key].items():
                        dset.attrs[attr_key] = attr_val
        else:
            dset = file[key]
            dset.resize(len(dset) + data_shape[0], axis=0)
            dset[-data_shape[0] :] = val
    file.close()
    return output_path

In [79]:
root_data = "/home/mathiane/LNENWork/data_RoFormerMIL_CaA1CaA2"
os.makedirs(root_data, exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "CaA1"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "CaA2"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "CaA1"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "CaA2"), exist_ok=True)

for tneid in set(BTProjArchCaA1CaA2["tne_id_c"]):
    print("Tneid = ", tneid)
    BTProjArchCaA1CaA2_c = BTProjArchCaA1CaA2[BTProjArchCaA1CaA2["tne_id_c"] == tneid]
    BTProjArchCaA1CaA2_c = BTProjArchCaA1CaA2_c.sort_values(by=["x","y"])
    BTProjArchCaA1CaA2_c["x_rank"] = BTProjArchCaA1CaA2_c["x"].rank(method="min")-1
    BTProjArchCaA1CaA2_c["y_rank"] = BTProjArchCaA1CaA2_c["y"].rank(method="min")-1
    BTProjArchCaA1CaA2_c["x_rank"] = BTProjArchCaA1CaA2_c["x_rank"].astype(int)
    BTProjArchCaA1CaA2_c["y_rank"] = BTProjArchCaA1CaA2_c["y_rank"].astype(int)
    
    BTProjArchCaA1CaA2_features =  BTProjArchCaA1CaA2_c.iloc[:,1:(1+128)]
    print("BT Features " , BTProjArchCaA1CaA2_features.shape)
    
    BTProjArchCaA1CaA2_coords = BTProjArchCaA1CaA2_c[["img_id_c", "x","y", "x_rank", "y_rank"]]
    BTProjArchCaA1CaA2_coords_for_export = BTProjArchCaA1CaA2_coords[["x_rank", "y_rank"]]
    #print(BTProjArchCaA1CaA2_coords.head())
    
    archetype_c = list(BTProjArchCaA1CaA2_c["arch"])[0]
    print(archetype_c)
    asset_dict_tumor = {   
        "coords": BTProjArchCaA1CaA2_coords_for_export.to_numpy(),
        "features": BTProjArchCaA1CaA2_features.to_numpy(dtype='float32'),
            }
    
    
    h5_file_tumor = os.path.join(root_data, "features_h5_files", archetype_c, tneid + ".h5" )
    save_hdf5(h5_file_tumor, asset_dict_tumor, attr_dict=None, mode = "w")
    
    coords_tumor_filename = os.path.join(root_data, "coords_table", archetype_c, tneid + "coords.csv" )
    BTProjArchCaA1CaA2_coords.to_csv(coords_tumor_filename, index=False )
    
    print("___________________________________________________ \n")

Tneid =  TNE0079
BT Features  (11168, 128)
CaA2
___________________________________________________ 

Tneid =  TNE0881
BT Features  (10283, 128)
CaA2
___________________________________________________ 

Tneid =  TNE0983
BT Features  (14032, 128)
CaA1
___________________________________________________ 

Tneid =  TNE0009
BT Features  (45150, 128)
CaA1
___________________________________________________ 

Tneid =  TNE0876
BT Features  (35759, 128)
CaA2
___________________________________________________ 

Tneid =  TNE0230
BT Features  (12171, 128)
CaA2
___________________________________________________ 

Tneid =  TNE1414
BT Features  (6773, 128)
CaA2
___________________________________________________ 

Tneid =  TNE0408
BT Features  (9398, 128)
CaA1
___________________________________________________ 

Tneid =  TNE0956
BT Features  (15796, 128)
CaA1
___________________________________________________ 

Tneid =  TNE0887
BT Features  (20404, 128)
CaA2
____________________________________

In [80]:
# Create slides_format slides
case_id = []
slide_id = []
label_id = []
label = []
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "CaA1")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("CaA1")
        label.append("CaA1")
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "CaA2")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("CaA2")
        label.append("CaA2")
slides_format = pd.DataFrame({"case_id":case_id,
                              "slide_id": slide_id,
                              "label_id": label_id,
                              "label": label})
slides_format.head()

,case_id,slide_id,label_id,label
0,TNE1794,TNE1794,CaA1,CaA1
1,TNE0993,TNE0993,CaA1,CaA1
2,TNE1448,TNE1448,CaA1,CaA1
3,TNE0889,TNE0889,CaA1,CaA1
4,TNE0001,TNE0001,CaA1,CaA1


In [81]:
slides_format.shape

(133, 4)

In [84]:
os.makedirs( os.path.join(root_data, "dataRoFormer"), exist_ok=True)

In [85]:
slides_format.to_csv( os.path.join(root_data, "dataRoFormer", "slides_format.csv"))

In [89]:
## Organiza dataRoFormer folder
os.makedirs( os.path.join(root_data, "dataRoFormer", "features" ), exist_ok=True)
os.makedirs( os.path.join(root_data, "dataRoFormer", "features" , "h5_files_grid" ), exist_ok=True)
os.makedirs( os.path.join(root_data, "dataRoFormer", "splits" ), exist_ok=True)
os.makedirs( os.path.join(root_data, "dataRoFormer", "features", "h5_files_grid" ), exist_ok=True)


FileExistsError: [Errno 17] File exists: '/home/mathiane/LNENWork/data_RoFormerMIL_CaA1CaA2/dataRoFormer/features/h5_files_grid'

In [90]:
os.listdir(os.path.join(root_data, "features_h5_files", "CaA2")
           )

['TNE1430.h5',
 'TNE0228.h5',
 'TNE0881.h5',
 'TNE0080.h5',
 'TNE1080.h5',
 'TNE0879.h5',
 'TNE0406.h5',
 'TNE0952.h5',
 'TNE1075.h5',
 'TNE0898.h5',
 'TNE1008.h5',
 'TNE0229.h5',
 'TNE0479.h5',
 'TNE0015.h5',
 'TNE0798.h5',
 'TNE1093.h5',
 'TNE1446.h5',
 'TNE0876.h5',
 'TNE1452.h5',
 'TNE0809.h5',
 'TNE1367.h5',
 'TNE0079.h5',
 'TNE1414.h5',
 'TNE0965.h5',
 'TNE0231.h5',
 'TNE1427.h5',
 'TNE0963.h5',
 'TNE0230.h5',
 'TNE1442.h5',
 'TNE1435.h5',
 'TNE0979.h5',
 'TNE1508.h5',
 'TNE0986.h5',
 'TNE1102.h5',
 'TNE0007.h5',
 'TNE1094.h5',
 'TNE1019.h5',
 'TNE0387.h5',
 'TNE1097.h5',
 'TNE0008.h5',
 'TNE0988.h5',
 'TNE1405.h5',
 'TNE0077.h5',
 'TNE1499.h5',
 'TNE0966.h5',
 'TNE0980.h5',
 'TNE0954.h5',
 'TNE0987.h5',
 'TNE0953.h5',
 'TNE1365.h5',
 'TNE1429.h5',
 'TNE0308.h5',
 'TNE0340.h5',
 'TNE0341.h5',
 'TNE0031.h5',
 'TNE0873.h5',
 'TNE0081.h5',
 'TNE0884.h5',
 'TNE0887.h5',
 'TNE1007.h5',
 'TNE1903.h5']